# Advanced Simulation Modeling

contact: annaszczurek2@gmail.com

## 02 CASE STUDY - MACHINES AND TOOLS

- 6 machines
- 5 repair tools
- trouble-free operation time for the machine ~ Exponential Dist with exp = 75min
- tools transportation time
  - 1st scenario: $t_i=i*2$
  - 2nd scenario: 3 minutes
- repair time ~ Erlang Dist with k=3 and exp = 15 minutes


## SOLUTION

### 1. Parameters

In [1]:
import numpy as np

In [2]:
# number of machines
m = 6 

# avgerage working time with no damage
avg_working_time = 75 # minutes

avg_repair_time = 15 # minutes

# number of repairing tools
n = 5

# time horizon
horizon = 30 # days

# simulation repeats
iterations = 10   

### 2. Model

Events - vectors that are designed to control the simulation state:

- moments of occurrence of subsequent events
- machines status:
    - `W` - operating 
    - `Q` - waiting for the tools 
    - `R` - is being repaired
- inactivity time


In [26]:
def model(horizon, avg_working_time, avg_repair_time, m, n, setup, events_distribution):
    # setup - 1st scenario = "L" ; second scenario = "S"
    
    # time horizon in minutes
    horizon = horizon * 24 * 60 
    
    # events - vector that changes the state of the simulation
    if events_distribution == "exp":
        events = list(np.random.exponential(avg_working_time, m))
    elif events_distribution == "pois":
        events = list(np.random.poisson(avg_working_time, m))
    
    # current machine status
    status = ["W"] * m

    # t_start - determines the beginning of machine inactivity
    t_start = [0] * m

    # t_cum - cumulative idle time of the machine
    t_cum = [0] * m

    # tools_loc tools location - machine id or '-1' for standby
    tools_loc = [-1] * n

    # tools_occupied occupation time for a tool (per machine)
    tools_occupied = [0] * n
    
    # simulation clock - the nearest task to be performed
    t = min(events)
    
    # start simulation - iterating over events
    while t <= horizon:
        
        # if tools are not occupied - move them to the workshop (standby position)
        for i in range(n):
            if tools_occupied[i] <= t:
                tools_loc[i] = -1

        # choose machine based on current event
        machine = events.index(t)
        
        
#         Event is always associated with the machine.
#         If the event-machine is working ("W"), the next state might be:
#           - "R" for being repaired if there is an available tool
#           - "Q" as "Queue" if currently there are no tools available
          
#         Steps for working machines ("W"):
#           1) update t_start vector with the time (t) when machine stopped working
#           2) check if there is any tool available
#              if not:
#                2.1) update machine status with "Q" as it's waiting in a queue for the tool
#                2.2) update events vector with the time the machine needs to wait for 
#                     the first available tool
#              if there is an available tool:
#                2.1) update machine status with "R" as it's being repaired
#                2.2) update events vector with time estimate needed for repairing the machine 
#                     - depending on the setup (time needed for tool transportation + repair)
#                2.3) update tools_loc vector with the index of the machine that is being repaired
#                2.4) update tools_occupied vector with the time needed for the repair 
#                     (2*transportation + repair)
        
        if status[machine] == "W":
            t_start[machine] = t
            tools = - 1
            for i in range(n):
                if tools_loc[i] == -1:
                    tools = i
                    break
            if tools == -1 : # if no tools available
                status[machine] = "Q" # change machine status to queue
                events[machine] = min(tools_occupied)
            else:
                status[machine] = "R"
                if setup == "L":
                    transport_time = 2 * (1 + machine) # adding 1 because python indexing starts from 0
                elif setup == "S":
                    transport_time =  3
                else:
                    print("Please choose one of the following: 'L' or 'S'!")
                    break
                repair_time = np.random.gamma(3, avg_repair_time/3)
                events[machine] += repair_time + transport_time
                tools_loc[tools] = machine
                tools_occupied[tools] += repair_time + 2 * transport_time

        
#         Steps for machine waiting for the tool in a queue ("Q")
#           1) select an available tool
#           2) update machine status with "R" as it's being repaired
#           3) update events vector with time estimate needed for repairing the machine 
#              - depending on the setup (time needed for tool transportation + repair)
#           4) update tools_loc vector with the index of the machine that is being repaired
#           5) update tools_occupied vector with the time needed for the repair 
#              (2*transportation + repair)

        elif status[machine] == "Q":
            for i in range(n):
                if tools_loc[i] == -1:
                    tools = i
                    break
            status[machine] = "R"
            if setup == "L":
                transport_time = 2 * (1 + machine)
            elif setup == "G":
                transport_time =  3
            else:
                print("Please choose one of the following: 'L' or 'S'!")
                break
            repair_time = np.random.gamma(3, avg_repair_time/3)
            events[machine] += repair_time + transport_time
            tools_loc[tools] = machine
            tools_occupied[tools] += repair_time + 2 * transport_time 
            
        
#         Steps for machine being repaired ("R")
#           1) update machine status with "W" for working
#           3) update events vector with the next inactivity time estimate
#           4) update t_cum with machine inactivity (t - t_start[machine])
            
        else:
            status[machine] = "W"
            events[machine] += np.random.exponential(avg_working_time)
            t_cum[machine] += t - t_start[machine]
        
        # new nearest event
        t = min(events)
        
    # result - a list of cumulated inactivity time for each of the machines
    return (t_cum)

### 3. `Run()` function

In [27]:
def run_model (iterations, horizon, avg_working_time, avg_repair_time, m, n, setup, events_distribution):
    avg_t_cum = []
    for i in range (iterations):
        print(f"iteration no {i+1}")
        avg_t_cum.append(model(horizon, avg_working_time, avg_repair_time, m, n, setup, events_distribution))
    return list(map(np.mean, np.transpose(avg_t_cum)))

### 3. Simulation results

- exponential distriubution

In [28]:
distribution = "exp"

In [34]:
avg_t_cum_exp_l = run_model(iterations, horizon, avg_working_time, avg_repair_time, m, n, "L", distribution)
avg_t_cum_exp_s = run_model(iterations, horizon, avg_working_time, avg_repair_time, m, n, "S", distribution)

iteration no 1
iteration no 2
iteration no 3
iteration no 4
iteration no 5
iteration no 6
iteration no 7
iteration no 8
iteration no 9
iteration no 10
iteration no 1
iteration no 2
iteration no 3
iteration no 4
iteration no 5
iteration no 6
iteration no 7
iteration no 8
iteration no 9
iteration no 10


In [35]:
print(np.mean(avg_t_cum_exp_l))
print(np.mean(avg_t_cum_exp_s))

9790.639130588568
8337.559267304867


- poisson distribution

In [31]:
distribution = "pois"

In [32]:
avg_t_cum_weib_l = run_model(iterations, horizon, avg_working_time, avg_repair_time, m, n, "L", distribution)
avg_t_cum_weib_s = run_model(iterations, horizon, avg_working_time, avg_repair_time, m, n, "S", distribution)

iteration no 1
iteration no 2
iteration no 3
iteration no 4
iteration no 5
iteration no 6
iteration no 7
iteration no 8
iteration no 9
iteration no 10
iteration no 1
iteration no 2
iteration no 3
iteration no 4
iteration no 5
iteration no 6
iteration no 7
iteration no 8
iteration no 9
iteration no 10


In [33]:
print(np.mean(avg_t_cum_weib_l))
print(np.mean(avg_t_cum_weib_s))

9816.705588465544
8278.012025313017
